<a href="https://colab.research.google.com/github/Catatau52/Estudos_Data_Science/blob/main/Challenge_DS_Week01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Criando Sessão do Spark

In [164]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [165]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Regressão com Spark") \
    .getOrCreate()

spark

#Montando o Drive e carregando dados

In [166]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [167]:
import zipfile

In [168]:
zipfile.ZipFile('/content/drive/MyDrive/curso-spark/Challenge-DS/semana-1.zip', 'r').extractall('/content/drive/MyDrive/curso-spark/Challenge-DS')

In [169]:
dados = spark.read.json(
    '/content/drive/MyDrive/curso-spark/Challenge-DS/dataset_bruto.json'
)

#Explorando Dados

In [170]:
dados.count()

89083

In [171]:
dados.show(5,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

Checando os tipos de variáveis.

In [172]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

#Filtrando os dados

In [173]:
dados\
    .select('anuncio.*')\
    .show(5,truncate=False)

+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                        |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|0    |[]    

In [174]:
anuncio = dados.select('anuncio.*')

In [175]:
anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [176]:
anuncio\
    .select('tipo_anuncio')\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



In [177]:
anuncio\
    .select('tipo_unidade')\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



Podemos elimiar o uso do tipo comercial, por não ter tanta influência.

In [178]:
anuncio\
    .select('tipo_uso')\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [179]:
anuncio\
    .filter("tipo_uso == 'Residencial'").show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|     [120]|    [120]|      [0]|[Portão eletrônic...|{Campo Grande, 23...|ac6cb348-69d6-45a...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{80, null, Venda...|
|    0|       [3]|      [3]|      [0]|                  []|{São Cristóvão, 2...|e032b908-ef42-4d4...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{0, 0, Venda, 50...|
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-4

In [180]:
anuncio_uso_residencial = anuncio.filter("tipo_uso == 'Residencial'")

In [181]:
anuncio\
    .filter("tipo_unidade == 'Apartamento'").show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [182]:
anuncio_tipo_apartamento = anuncio.filter("tipo_unidade == 'Apartamento'")

#Convertendo e tratando dados

Transformando os dados das colunas "quartos", "suites", "banheiros", "vaga", "area_total" e "area_util" de listas para inteiros.

In [183]:
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, DoubleType, ArrayType

 Checando se as colunas tem apenas inteiros.

In [184]:
anuncio\
    .select(f.size(f.col('quartos')).alias('quartos'))\
    .groupBy('quartos')\
    .count()\
    .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|88120|
|      2|  160|
|      0|  803|
+-------+-----+



Atribuindo as carterísticas e contando cada uma

In [185]:
caracteristicas = ["quartos", "suites", "banheiros", "vaga", "area_total", "area_util"]

In [186]:
for col in caracteristicas:
    anuncio\
        .select(f.size(f.col(col)).alias(col))\
        .groupBy(col)\
        .count()\
        .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|88120|
|      2|  160|
|      0|  803|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|79431|
|     2|  103|
|     0| 9549|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|88798|
|        2|  112|
|        0|  173|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|84037|
|   2|   94|
|   0| 4952|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|76488|
|         0|12595|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|88846|
|        2|  224|
|        0|   13|
+---------+-----+



In [187]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

Retirando os dados de dentro da lista

In [188]:
anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [189]:
#list comprehension python
anuncio\
      .select ([f.col(c)[0].alias(c)if c in caracteristicas else c for c in anuncio.columns])\
      .show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|      null|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

Atualizando os dados

In [190]:
anuncio = anuncio.select ([f.col(c)[0].alias(c)if c in caracteristicas else c for c in anuncio.columns])
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

In [191]:
anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|      null|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

Contando quantos "null" existem em cada coluna

In [192]:
anuncio\
    .select([f.count(f.when(f.isnull(c), True)).alias(c) for c in anuncio.columns])\
    .show()

+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+
|andar|area_total|area_util|banheiros|caracteristicas|endereco| id|quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso|vaga|valores|
+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+
|    0|     12595|       13|      173|              0|       0|  0|    803|  9549|           0|           0|       0|4952|      0|
+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+



Convertendo para o tipo Inteiro

In [193]:
#caracteristicas = ["quartos", "suites", "banheiros", "vaga", "area_total", "area_util"]
anuncio = anuncio\
    .withColumn('quartos', anuncio['quartos'].cast(IntegerType()))\
    .withColumn('suites', anuncio['suites'].cast(IntegerType()))\
    .withColumn('banheiros', anuncio['banheiros'].cast(IntegerType()))\
    .withColumn('vaga', anuncio['vaga'].cast(IntegerType()))\
    .withColumn('area_total', anuncio['area_total'].cast(IntegerType()))\
    .withColumn('area_util', anuncio['area_util'].cast(IntegerType()))
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 

Preenchendo dados faltantes com 0

In [194]:
#anuncio2 = anuncio2.fillna(value=0,subset=['banheiros','quartos', 'suites','vaga'])
#anuncio2 = anuncio2.na.fill(value='0',subset=['area_total','area_util'])
anuncio = anuncio.na.fill(value=0,subset=['area_total','area_util','banheiros','quartos', 'suites','vaga'])
anuncio.show(10)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|         0|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|         0|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|     0|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

Checando se há valores nulos

In [195]:
anuncio.filter(anuncio.area_total.isNull()).show()

+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+
|andar|area_total|area_util|banheiros|caracteristicas|endereco| id|quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso|vaga|valores|
+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+
+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+



In [196]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 

In [197]:
from pyspark.sql.functions import col

In [198]:
cols = ("andar","caracteristicas","endereco","id","tipo_anuncio","tipo_unidade","tipo_uso","valores")

anuncio2 = anuncio.drop(*cols)
anuncio2.printSchema()

root
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: integer (nullable = true)



In [199]:
anuncio2.show(5)

+----------+---------+---------+-------+------+----+
|area_total|area_util|banheiros|quartos|suites|vaga|
+----------+---------+---------+-------+------+----+
|         0|       16|        0|      0|     0|   1|
|         0|       14|        0|      0|     0|   0|
|      1026|     1026|        0|      0|     0|   0|
|       120|      120|        0|      0|     0|   0|
|         3|        3|        0|      0|     0|   0|
+----------+---------+---------+-------+------+----+
only showing top 5 rows



#Analisando a coluna endereço
A equipe de ciência de dados nos solicitou que apenas as informações sobre bairro e zona da cidade fossem extraídas.

Então, vamos analisar a coluna endereco e transformar apenas as informações sobre bairro e zona em colunas de nosso DataFrame.

In [200]:
anuncio_endereco = anuncio.select('endereco')
anuncio_endereco.show(5,truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+
|endereco                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------+
|{Centro, 20061003, Rio de Janeiro, Rio de Janeiro, -22.906082, -43.18671, BR, Rua Buenos Aires, Zona Central}                |
|{Centro, 20051040, Rio de Janeiro, Rio de Janeiro, -22.902536, -43.18434, BR, Avenida Passos, Zona Central}                  |
|{Maria da Graça, 20785360, Rio de Janeiro, Rio de Janeiro, -22.885516, -43.269083, BR, Rua Luís de Brito, Zona Norte}        |
|{Campo Grande, 23095071, Rio de Janeiro, Rio de Janeiro, -22.885028, -43.561668, BR, Estrada Guandu do Sape, Zona Oeste}     |
|{São Cristóvão, 20931670, Rio de Janeiro, Rio de Janeiro, -22.885912, -43.219296, BR, Rua Monsenhor Man

In [201]:
anuncio_endereco.printSchema()

root
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)



In [202]:
anuncio_endereco = anuncio_endereco.select("endereco.bairro","endereco.zona")
anuncio_endereco.show(5,truncate=False)

+--------------+------------+
|bairro        |zona        |
+--------------+------------+
|Centro        |Zona Central|
|Centro        |Zona Central|
|Maria da Graça|Zona Norte  |
|Campo Grande  |Zona Oeste  |
|São Cristóvão |Zona Norte  |
+--------------+------------+
only showing top 5 rows



In [203]:
anuncio_endereco.printSchema()

root
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



#Analisando os tipos VENDA

In [204]:
anuncio_valores = anuncio.select(['valores.condominio','valores.iptu','valores.tipo','valores.valor'])
anuncio_valores.show(5,truncate=False)

+----------+------+-------+-------+
|condominio|iptu  |tipo   |valor  |
+----------+------+-------+-------+
|[260]     |[107] |[Venda]|[10000]|
|[260]     |[107] |[Venda]|[10000]|
|[null]    |[1613]|[Venda]|[10000]|
|[80]      |[null]|[Venda]|[10000]|
|[0]       |[0]   |[Venda]|[5000] |
+----------+------+-------+-------+
only showing top 5 rows



In [205]:
tipos = ['condominio','iptu','tipo','valor']
anuncio_valores\
      .select ([f.col(c)[0].alias(c)if c in tipos else c for c in anuncio_valores.columns])\
      .show(5)

+----------+----+-----+-----+
|condominio|iptu| tipo|valor|
+----------+----+-----+-----+
|       260| 107|Venda|10000|
|       260| 107|Venda|10000|
|      null|1613|Venda|10000|
|        80|null|Venda|10000|
|         0|   0|Venda| 5000|
+----------+----+-----+-----+
only showing top 5 rows



In [206]:
anuncio_valores = anuncio_valores\
      .select ([f.col(c)[0].alias(c)if c in tipos else c for c in anuncio_valores.columns])

In [207]:
anuncio_valores.printSchema()

root
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



convertendo os dados de Valores

In [208]:
anuncio_valores\
    .select([f.count(f.when(f.isnull(c), True)).alias(c) for c in anuncio_valores.columns])\
    .show()

+----------+-----+----+-----+
|condominio| iptu|tipo|valor|
+----------+-----+----+-----+
|      5943|10436|   0|   28|
+----------+-----+----+-----+



In [209]:
anuncio_valores.select('condominio').count()

89083

In [210]:
anuncio_valores = anuncio_valores\
    .withColumn('condominio', anuncio_valores['condominio'].cast(IntegerType()))\
    .withColumn('iptu', anuncio_valores['iptu'].cast(IntegerType()))\
    .withColumn('valor', anuncio_valores['valor'].cast(IntegerType()))
anuncio_valores.printSchema()

root
 |-- condominio: integer (nullable = true)
 |-- iptu: integer (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: integer (nullable = true)



In [211]:
anuncio_valores.show(5)

+----------+----+-----+-----+
|condominio|iptu| tipo|valor|
+----------+----+-----+-----+
|       260| 107|Venda|10000|
|       260| 107|Venda|10000|
|      null|1613|Venda|10000|
|        80|null|Venda|10000|
|         0|   0|Venda| 5000|
+----------+----+-----+-----+
only showing top 5 rows



In [212]:
anuncio_valores = anuncio_valores.na.fill(value=0,subset=['condominio','iptu','valor'])
anuncio_valores.show(10)

+----------+----+-----+-----+
|condominio|iptu| tipo|valor|
+----------+----+-----+-----+
|       260| 107|Venda|10000|
|       260| 107|Venda|10000|
|         0|1613|Venda|10000|
|        80|   0|Venda|10000|
|         0|   0|Venda| 5000|
|       223|  87|Venda|18500|
|       245|   0|Venda|15000|
|         0|   0|Venda|15000|
|       250|   0|Venda|20000|
|         0|   0|Venda|20000|
+----------+----+-----+-----+
only showing top 10 rows



In [213]:
anuncio_valores\
    .select([f.count(f.when(f.isnull(c), True)).alias(c) for c in anuncio_valores.columns])\
    .show()

+----------+----+----+-----+
|condominio|iptu|tipo|valor|
+----------+----+----+-----+
|         0|   0|   0|    0|
+----------+----+----+-----+



In [214]:
anuncio_valores.filter(anuncio_valores.tipo == "Venda").show(5)

+----------+----+-----+-----+
|condominio|iptu| tipo|valor|
+----------+----+-----+-----+
|       260| 107|Venda|10000|
|       260| 107|Venda|10000|
|         0|1613|Venda|10000|
|        80|   0|Venda|10000|
|         0|   0|Venda| 5000|
+----------+----+-----+-----+
only showing top 5 rows



In [215]:
anuncio_valores.filter(anuncio_valores.tipo == "Aluguel").show(5)

+----------+----+-------+-----+
|condominio|iptu|   tipo|valor|
+----------+----+-------+-----+
|       500| 117|Aluguel|  500|
|       295| 107|Aluguel|  250|
|       600| 126|Aluguel|  100|
|         0| 115|Aluguel|  550|
|       200|   0|Aluguel|  800|
+----------+----+-------+-----+
only showing top 5 rows



#Salvando os dados

CSV

In [220]:
%%time
anuncio_valores_csv = anuncio_valores
anuncio_valores_csv.write.option("header", True).csv("/content/drive/MyDrive/curso-spark/anuncio_valores_csv")
#anuncio_valores_csv.write.format("csv").save("/content/drive/MyDrive/curso-spark/anuncio_valores_csv")

CPU times: user 27 ms, sys: 572 µs, total: 27.6 ms
Wall time: 3.2 s


In [217]:
#csvFile = spark.read.option("header",True).csv("/content/drive/MyDrive/curso-spark/")
#csvFile.show(5)

Parquet

In [219]:
%%time
anuncio_valores_parquet = anuncio_valores
anuncio_valores_parquet.write.parquet(path='/content/drive/MyDrive/curso-spark/anuncio_valores_parquet', mode='overwrite')

CPU times: user 33.9 ms, sys: 2.89 ms, total: 36.7 ms
Wall time: 4.23 s
